In [12]:
# nuclio: ignore
import nuclio

In [13]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-base"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/ml-base'


In [14]:
import numpy as np
import pandas as pd
from mlrun.execution import MLClientCtx

def load_dataset(
        context: MLClientCtx,
        dataset: str,
        name: str = '',
        file_ext: str = 'parquet',
        params: dict = {}
) -> None:
    """Loads a scikit-learn toy dataset for classification or regression

    The following datasets are available ('name' : desription):

        'boston'          : boston house-prices dataset (regression)
        'iris'            : iris dataset (classification)
        'diabetes'        : diabetes dataset (regression)
        'digits'          : digits dataset (classification)
        'linnerud'        : linnerud dataset (multivariate regression)
        'wine'            : wine dataset (classification)
        'breast_cancer'   : breast cancer wisconsin dataset (classification)

    The scikit-learn functions return a data bunch including the following items:
    - data              the features matrix
    - target            the ground truth labels
    - DESCR             a description of the dataset
    - feature_names     header for data

    The features (and their names) are stored with the target labels in a DataFrame.

    For further details see https://scikit-learn.org/stable/datasets/index.html#toy-datasets

    :param context:    function execution context
    :param dataset:    name of the dataset to load
    :param name:       artifact name (defaults to dataset)
    :param file_ext:   output file_ext: parquet or csv
    :param params:     params of the sklearn load_data method
    """
    dataset = str(dataset)
    # reach into module and import the appropriate load_xxx function
    pkg_module = 'sklearn.datasets'
    fname = f'load_{dataset}'

    pkg_module = __import__(pkg_module, fromlist=[fname])
    load_data_fn = getattr(pkg_module, fname)

    data = load_data_fn(**params)
    feature_names = data['feature_names']

    # create the toy dataset
    xy = np.concatenate([data['data'], data['target'].reshape(-1, 1)], axis=1)
    if hasattr(feature_names, 'append'):
        # its a list
        feature_names.append('labels')
    else:
        # its an array
        feature_names = np.append(feature_names, 'labels')
    df = pd.DataFrame(data=xy, columns=feature_names)

    # log and upload the dataset
    context.log_dataset(name or dataset, df=df, format=file_ext, index=False)

In [15]:
# nuclio: end-code

### mlconfig

In [16]:
from mlrun import mlconf
import os

mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [17]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dataset")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dataset"
fn.spec.description = "load a toy dataset from scikit-learn"
fn.metadata.categories = ["data-source", "ml"]
fn.metadata.labels = {"author": "yjb", "framework": "sklearn"}
fn.export("function.yaml")

[mlrun] 2020-05-14 17:48:05,723 function spec saved to path: function.yaml


## tests

In [18]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc('nfsvol', 'nfsvol', '/home/jovyan/data'))

In [19]:
from mlrun import NewTask    

task_params = {
    "name"   : "tasks load toy dataset", 
    "params" : {"dataset"     : "wine"}}

### run remotely

In [20]:
run = fn.run(NewTask(**task_params), artifact_path=mlconf.artifact_path)

[mlrun] 2020-05-14 17:48:15,160 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 17:48:15,161 starting run tasks load toy dataset uid=06abcf9484e24fb3b7bae792cd41c4fc  -> http://mlrun-api:8080
[mlrun] 2020-05-14 17:48:15,258 Job is running in the background, pod: tasks-load-toy-dataset-cc2mw
[mlrun] 2020-05-14 17:48:18,867 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 17:48:18,883 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 17:48:19,229 log artifact wine at /User/artifacts/wine.parquet, size: 18454, db: Y

[mlrun] 2020-05-14 17:48:19,264 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...cd41c4fc,0,May 14 17:48:18,completed,tasks load toy dataset,host=tasks-load-toy-dataset-cc2mwkind=jobowner=adminv3io_user=admin,,dataset=wine,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run 06abcf9484e24fb3b7bae792cd41c4fc  , !mlrun logs 06abcf9484e24fb3b7bae792cd41c4fc 
[mlrun] 2020-05-14 17:48:21,460 run executed, status=completed


### or locally

In [21]:
from mlrun import run_local

In [22]:
for dataset in ["wine", "iris", "breast_cancer"]:
    run_local(handler=load_dataset,
              inputs={"dataset": dataset}, artifact_path=mlconf.artifact_path)

[mlrun] 2020-05-14 17:48:25,731 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 17:48:25,731 starting run mlrun-045faa-load_dataset uid=a28fa3189ffe41d19aafd71e071af7b9  -> http://mlrun-api:8080
[mlrun] 2020-05-14 17:48:25,754 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 17:48:25,867 log artifact wine at /User/artifacts/wine.parquet, size: 18006, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...071af7b9,0,May 14 17:48:25,completed,mlrun-045faa-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-f597579bb-l8hbz,dataset,,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run a28fa3189ffe41d19aafd71e071af7b9 --project default , !mlrun logs a28fa3189ffe41d19aafd71e071af7b9 --project default
[mlrun] 2020-05-14 17:48:25,937 run executed, status=completed
[mlrun] 2020-05-14 17:48:25,945 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 17:48:25,946 starting run mlrun-8818fb-load_dataset uid=852572c520cb4a3884a9b3c3184a4d24  -> http://mlrun-api:8080
[mlrun] 2020-05-14 17:48:25,969 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 17:48:26,040 log artifact iris at /User/artifacts/iris.parquet, size: 5049, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...184a4d24,0,May 14 17:48:25,completed,mlrun-8818fb-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-f597579bb-l8hbz,dataset,,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run 852572c520cb4a3884a9b3c3184a4d24 --project default , !mlrun logs 852572c520cb4a3884a9b3c3184a4d24 --project default
[mlrun] 2020-05-14 17:48:26,088 run executed, status=completed
[mlrun] 2020-05-14 17:48:26,105 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 17:48:26,105 starting run mlrun-aa809c-load_dataset uid=4c1d6a176c794d538dcb89312c56da45  -> http://mlrun-api:8080
[mlrun] 2020-05-14 17:48:26,125 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 17:48:26,301 log artifact breast_cancer at /User/artifacts/breast_cancer.parquet, size: 146754, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...2c56da45,0,May 14 17:48:26,completed,mlrun-aa809c-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-f597579bb-l8hbz,dataset,,,breast_cancer


to track results use .show() or .logs() or in CLI: 
!mlrun get run 4c1d6a176c794d538dcb89312c56da45 --project default , !mlrun logs 4c1d6a176c794d538dcb89312c56da45 --project default
[mlrun] 2020-05-14 17:48:26,379 run executed, status=completed
